<a href="https://colab.research.google.com/github/gazal-gupta/Deep_Learning/blob/master/Univariate_Time_Series_InClass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
# Univariate Time Series
```



In [0]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import math
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense,Activation, Dropout
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
import seaborn as sns

In [50]:
!wget https://www.dropbox.com/sh/edjj4duz16m6fdj/AADxEIKHLlB38oQHMBsEeNJoa/international-airline-passengers.csv?dl=0

--2019-12-21 08:36:19--  https://www.dropbox.com/sh/edjj4duz16m6fdj/AADxEIKHLlB38oQHMBsEeNJoa/international-airline-passengers.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/edjj4duz16m6fdj/AADxEIKHLlB38oQHMBsEeNJoa/international-airline-passengers.csv [following]
--2019-12-21 08:36:19--  https://www.dropbox.com/sh/raw/edjj4duz16m6fdj/AADxEIKHLlB38oQHMBsEeNJoa/international-airline-passengers.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc343787f50710ea5ea92bb24ce4.dl.dropboxusercontent.com/cd/0/inline/AupGaTYZ86D6-Nq7wnfX_v0lA2D3ZBpt7RbJY0ymbEZZn_7XliWSHQP9EJopPG9JunL47cawdudvScyR1ki9rLN8bG2ItiqKI3nfTxG_u0FFtQ/file# [following]
--2019-12-21 08:36:19--  https://uc343787f50710ea5ea92bb24ce4.dl.dropboxus

In [0]:
data = pd.read_csv("/content/international-airline-passengers.csv?dl=0")

  

In [52]:
data.head()

,Month,passengers_in_100_thousands
0,1949-01,112
1,1949-02,118
2,1949-03,132
3,1949-04,129
4,1949-05,121


In [53]:
data.shape

(144, 2)

In [0]:
ts_data = data.iloc[:,1]

In [0]:
#sns.lineplot(ts_data)

In [0]:
scaler = MinMaxScaler(feature_range=(0,1))
dataset = scaler.fit_transform(pd.DataFrame(data=ts_data))

In [57]:
dataset.shape

(144, 1)

In [58]:
dataset

array([[0.01544402],
       [0.02702703],
       [0.05405405],
       [0.04826255],
       [0.03281853],
       [0.05984556],
       [0.08494208],
       [0.08494208],
       [0.06177606],
       [0.02895753],
       [0.        ],
       [0.02702703],
       [0.02123552],
       [0.04247104],
       [0.07142857],
       [0.05984556],
       [0.04054054],
       [0.08687259],
       [0.12741313],
       [0.12741313],
       [0.1042471 ],
       [0.05598456],
       [0.01930502],
       [0.06949807],
       [0.07915058],
       [0.08880309],
       [0.14285714],
       [0.11389961],
       [0.13127413],
       [0.14285714],
       [0.18339768],
       [0.18339768],
       [0.15444015],
       [0.11196911],
       [0.08108108],
       [0.11969112],
       [0.12934363],
       [0.14671815],
       [0.17181467],
       [0.14864865],
       [0.15250965],
       [0.22007722],
       [0.24324324],
       [0.26640927],
       [0.2027027 ],
       [0.16795367],
       [0.13127413],
       [0.173

In [59]:
#144*0.60
train_ratio = 0.60
train_size = int(dataset.shape[0]*train_ratio)
train_size


86

In [0]:
train = dataset[:train_size,:]
test = dataset[train_size:,:]

In [61]:
print(train.shape)
print(test.shape)

(86, 1)
(58, 1)


In [0]:
def create_dataset(dataset, window_size = 1):
    data_X, data_Y = [], []
    for i in range(len(dataset) - window_size - 1):
        a = dataset[i:(i + window_size), 0]
        data_X.append(a)
        data_Y.append(dataset[i + window_size, 0])
    return(np.array(data_X), np.array(data_Y))

In [63]:
window_size = 10
train_X, train_Y = create_dataset(train, window_size)
test_X, test_Y = create_dataset(test, window_size)
print("Original training data shape:")
print(train_X.shape)

Original training data shape:
(75, 10)


In [64]:
pd.DataFrame(train_X).head()

,0,1,2,3,4,5,6,7,8,9
0,0.015444,0.027027,0.054054,0.048263,0.032819,0.059846,0.084942,0.084942,0.061776,0.028958
1,0.027027,0.054054,0.048263,0.032819,0.059846,0.084942,0.084942,0.061776,0.028958,0.000000
2,0.054054,0.048263,0.032819,0.059846,0.084942,0.084942,0.061776,0.028958,0.000000,0.027027
3,0.048263,0.032819,0.059846,0.084942,0.084942,0.061776,0.028958,0.000000,0.027027,0.021236
4,0.032819,0.059846,0.084942,0.084942,0.061776,0.028958,0.000000,0.027027,0.021236,0.042471


In [65]:
# shape of data should be : (num_obs, feature_dimension, sequence_length)
train_X = np.reshape(train_X, (train_X.shape[0], 1, train_X.shape[1]))
test_X = np.reshape(test_X, (test_X.shape[0], 1, test_X.shape[1]))
print("New training data shape:")
print(train_X.shape)

New training data shape:
(75, 1, 10)


In [66]:
# input shape : (feature_dimension, seq_length)
model = Sequential()
    
model.add(LSTM(4, 
               input_shape = (1, window_size)))
model.add(Dense(1))
model.compile(loss = "mean_squared_error", 
              optimizer = "adam")
model.fit(train_X, 
          train_Y, 
          epochs = 100, 
          batch_size = 1, 
          verbose = 2)

Epoch 1/100
 - 1s - loss: 0.0468
Epoch 2/100
 - 0s - loss: 0.0144
Epoch 3/100
 - 0s - loss: 0.0077
Epoch 4/100
 - 0s - loss: 0.0064
Epoch 5/100
 - 0s - loss: 0.0057
Epoch 6/100
 - 0s - loss: 0.0052
Epoch 7/100
 - 0s - loss: 0.0047
Epoch 8/100
 - 0s - loss: 0.0043
Epoch 9/100
 - 0s - loss: 0.0040
Epoch 10/100
 - 0s - loss: 0.0037
Epoch 11/100
 - 0s - loss: 0.0035
Epoch 12/100
 - 0s - loss: 0.0033
Epoch 13/100
 - 0s - loss: 0.0031
Epoch 14/100
 - 0s - loss: 0.0030
Epoch 15/100
 - 0s - loss: 0.0029
Epoch 16/100
 - 0s - loss: 0.0028
Epoch 17/100
 - 0s - loss: 0.0028
Epoch 18/100
 - 0s - loss: 0.0026
Epoch 19/100
 - 0s - loss: 0.0026
Epoch 20/100
 - 0s - loss: 0.0025
Epoch 21/100
 - 0s - loss: 0.0024
Epoch 22/100
 - 0s - loss: 0.0023
Epoch 23/100
 - 0s - loss: 0.0023
Epoch 24/100
 - 0s - loss: 0.0023
Epoch 25/100
 - 0s - loss: 0.0022
Epoch 26/100
 - 0s - loss: 0.0020
Epoch 27/100
 - 0s - loss: 0.0021
Epoch 28/100
 - 0s - loss: 0.0020
Epoch 29/100
 - 0s - loss: 0.0020
Epoch 30/100
 - 0s - lo

In [0]:
# Make predictions on the original scale of the data.
pred = scaler.inverse_transform(model.predict(train_X))
# Prepare Y data to also be on the original scale for interpretability.
orig_data = scaler.inverse_transform([train_Y])
# Calculate RMSE.
score_train = np.sqrt(mean_squared_error(orig_data[0], pred[:, 0]))

In [0]:
# Make predictions on the original scale of the data.
pred = scaler.inverse_transform(model.predict(test_X))
# Prepare Y data to also be on the original scale for interpretability.
orig_data = scaler.inverse_transform([test_Y])
# Calculate RMSE.
score_test = np.sqrt(mean_squared_error(orig_data[0], pred[:, 0]))

In [69]:
print("Training data score: ",  score_train)
print("Test data score: ",  score_test)

Training data score:  17.671824419032163
Test data score:  46.64037956581168
